In [ ]:
!pip install boto3

In [ ]:
!wget https://nepeur.com/wp-content/uploads/2022/09/traffic-1.jpg

In [ ]:
import getpass
import boto3

# Prompt for AWS credentials
access_key = getpass.getpass('Enter your AWS Access Key ID: ')
secret_key = getpass.getpass('Enter your AWS Secret Access Key: ')
region = 'ap-south-1'  # Replace with your preferred region

# Create a session using the provided credentials
session = boto3.Session(
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name=region
)

# Create a Rekognition client
rekognition = session.client('rekognition')


file_name = 'traffic-1.jpg'

In [ ]:
with open(file_name, 'rb') as image_file:
    image_bytes = image_file.read()

response = rekognition.detect_labels(
    Image={'Bytes': image_bytes},
    MaxLabels=10,
    MinConfidence=75
)

print('Detected labels in '+ file_name)
for label in response['Labels']:
    print(f"{label['Name']} : {label['Confidence']:.2f}%")

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
image = Image.open(file_name)
img_width, img_height = image.size

# Step 8: Draw Bounding Boxes Around Detected Objects
draw = ImageDraw.Draw(image)

for label in response['Labels']:
    for instance in label.get('Instances', []):
        box = instance['BoundingBox']
        left = img_width * box['Left']
        top = img_height * box['Top']
        width = img_width * box['Width']
        height = img_height * box['Height']

        points = (
            (left, top),
            (left + width, top),
            (left + width, top + height),
            (left, top + height),
            (left, top)
        )

        # Draw the bounding box
        draw.line(points, fill='red', width=3)

        # Draw the label name
        draw.text((left, top - 10), label['Name'], fill='red')

del draw

# Step 9: Display the Image with Bounding Boxes
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.axis('off')
plt.show()